# **Setting Up**

In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5

In [ ]:
# Connect to Google Drive for files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd "/content/gdrive/MyDrive"

In [ ]:
# Important Stuff
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
import os
import gradio as gr
os.environ["OPENAI_API_KEY"] = ""

# **Reading the PDF**

In [ ]:
reader = PdfReader("/test/AS.pdf")

In [ ]:
# Read Data from Text:
raw_text = ''

# **CharacterTextSplitter**

In [ ]:
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text
# Split Text Data:
text_splitter = CharacterTextSplitter(
    separator= "\n",
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len
)

In [ ]:
texts = text_splitter.split_text(raw_text)

# **Embeddings**

In [ ]:
embeddings = OpenAIEmbeddings()

# **FAISS - Facebook A.I. Similarity Search**

In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)

##**LangChain**

In [ ]:
# Using ChatGPT-3.5-turbo for the document answering
llm = ChatOpenAI(temperature=0.0)

**Chain**

In [ ]:
# Question & Answer Chain
chain = load_qa_chain(llm = llm, chain_type="stuff")

# **Creating Function for our Questions**

In [ ]:
def ask_GPT(question):
    query = question
    docs = docsearch.similarity_search(query)
    response = chain.run(input_documents = docs, question=query)
    return response

In [ ]:
# Ask Question Here!
ask_GPT("")

'First ionisation energy is the energy required to remove one mole of electrons from one mole of gaseous atoms in their ground state to form one mole of gaseous ions with a single positive charge (cations). It is a measure of the strength of attraction between the positively charged nucleus and the valence electron that is to be removed. First ionisation energies generally increase across a period as the number of protons and electrostatic attraction between the valence electrons and the nucleus increases.'

# **Web Interface**

In [ ]:
demo = gr.Interface(
    fn = ask_GPT,
    inputs = gr.Textbox(lines = 2, placeholder="Enter your prompt: "),
    outputs = "text"
)
demo.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b5f878b89eadf4723b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
